# The algorithm as it stands

Take two catalogues as input:
- S3M is a simulated catalogue that contains a representative population of the solar system
- MPCOrb contains real data from actual detections

The goal is to combine the two to make a hybrid catalogue.

In order to do this we will:
1. Split catalogues into bins of absolute magnitude and for each bin
    1. Create a K-D Tree for the matching objects in the simulated catalogue (`tree`)
    2. Create a set of all objects that have already been assigned (`assigned`)
    3. For each object in the mpcorb catalogue with matching magnitudes:
        1. Set `k=100`
        2. Query the `tree` to find the nearest `k` objects in position within a distance `d_max`, save as `neighbours` and save length of `neighbours` as `count_nearby`
        3. Remove any objects that are already in `assigned`
        4. While `len(neighbours) < 1` and `count_nearby > 0` and `k < 500`
            1. `k += 100`
            2. Go back to step b
        5. If `len(neighbours) >= 1` then
            1. Find the object in `neighbours` with the closest velocity
            2. Save the pairing and add the object to `assigned`
        6. Otherwise
            1. Make a note that the real object has no match (so we will add it directly)
3. Output the list of pairings

In [1]:
from scipy.spatial import KDTree, cKDTree
import numpy as np
import pandas as pd

## Read in the data

In [2]:
s3m = pd.read_hdf("catalogues/s3m_propagated.h5", key="df")

In [3]:
mpcorb = pd.read_hdf("catalogues/mpcorb_propagated.h5", key="df")

In [4]:
mpcorb_xyz = np.array([mpcorb["x"].values, mpcorb["y"].values, mpcorb["z"].values]).T
s3m_xyz = np.array([s3m["x"].values, s3m["y"].values, s3m["z"].values]).T

## Split into absolute magnitude bins

In [33]:
%%timeit -n 1 -r 1

# loop every magnitude bin
magnitudes = np.arange(-2, 28)
for i in range(len(magnitudes) - 1):
    i = 14
    
    # get the matching simulated data and build a K-D Tree
    s3m_mag_mask = np.logical_and(s3m.H >= magnitudes[i], s3m.H < magnitudes[i + 1])
    s3m_id = s3m.id[s3m_mag_mask].values
    tree = cKDTree(s3m_xyz[s3m_mag_mask])
    
    # get the matching real data from MPCORB
    mpcorb_mag_mask = np.logical_and(mpcorb.H >= magnitudes[i], mpcorb.H <= magnitudes[i + 1])
    real_objects = mpcorb_xyz[mpcorb_mag_mask]
    
    print(len(s3m.H[s3m_mag_mask]), len(real_objects))
    
    # keep track of objects already assigned and a count of how many had no matches
    taken = []
    no_match_count = 0
    
    # iterate over every object in the real catalogue
    for obj in real_objects:
        
        # find the nearest 100 neighbours within 0.1 AU and mask further neighbours
        distances, inds = tree.query(obj, k=100, distance_upper_bound=0.1)
        distances, inds = distances[np.isfinite(distances)], inds[np.isfinite(distances)]
        
        ids = s3m_id[inds]
        
        # get only the options which haven't yet been assigned
        unassigned_options = np.setdiff1d(ids, taken, assume_unique=True)

        # if there are any matching objects
        if len(unassigned_options) > 0:
            
            # choose the one with the closest velocity
            match = unassigned_options[0]
            taken.append(match)
        else:
            no_match_count += 1
    
    print(len(taken), no_match_count)
            
    break

8404 5364
1910 3454
1.6 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
